## Paper LBO Model Example

Case Study Source: <http://www.streetofwalls.com/finance-training-courses/private-equity-training/paper-lbo-model-example/>


In [30]:
# Install dependencies
%pip install -r requirements.txt

  Using cached pyarrow-13.0.0.tar.gz (1.0 MB)
  Installing build dependencies ... \^C
anceled
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
# import libs
import polars as pl
import numpy_financial as np

#### 1. Entry Assumptions


In [78]:
# In the case of encapsulating data,
# why not use a class?
entry_multiple = 5
# As a monthly statement I guess just assume year 1 revenue/12 =  revenue per month
rev_y1 = 100 * 1000 * 1000 / 12
ebitda_margin = 0.4
debt_ratio = 0.6
equity_ratio = 0.4

ebitda_y1 = rev_y1 * ebitda_margin
price_paid = entry_multiple * ebitda_y1

# Debt & Equity Assumptions
debt = debt_ratio * price_paid
equity = equity_ratio * price_paid

# Income Statement Assumptions
rev_growth = 1.1
int_rate = 0.10
tax_rate = 0.4
capex = 0.15
working_capital = 5 * 1000 * 1000
d_and_a = 20 * 1000 * 1000 / 12

# Project Period
total_months = 12 * 6
months = [month for month in range(1, total_months + 1)]

#### 2. Income Statement


##### Build Income Statement Class


In [102]:
# Create the DataFrame for the income statement.
# Instead using a spreadsheet-like left-to-right approch,
# treating it as sans-index dataframe would be more benefitial,
# which means a top-to-bottom vertical structure.


class IncomeStatement:
    def __init__(
        self,
        year1_rev_monthly: float or int,
        rev_growth: float,
        ebitda_margin: float,
        int_rate: float,
        tax_rate: float,
        da: float or int,
        debt: float,
        total_months: int,
    ) -> None:
        self.year1_rev_monthly = year1_rev_monthly
        self.rev_growth = rev_growth
        self.ebitda_margin = ebitda_margin
        self.int_rate = int_rate
        self.tax_rate = tax_rate
        self.da = da
        self.debt = debt
        self.total_months = total_months

    def months(self) -> pl.DataFrame:
        return pl.DataFrame(
            data={
                "Month": [f"Month {month}" for month in range(1, self.total_months + 1)]
            }
        )

    def revenue(self) -> pl.DataFrame:
        revenue = []
        revenue_dict = {"Revenue": revenue}
        month = 0
        while month < self.total_months:
            if len(revenue) == 0:
                revenue.append(self.year1_rev_monthly)
                month += 1
            elif len(revenue) % 12 == 0:
                revenue.append(revenue[-1] * 1.1)
                month += 1
            else:
                revenue.append(revenue[-1])
                month += 1

        revenue = pl.DataFrame(data=revenue_dict)
        return self.months().with_columns(revenue)

    def ebitda(self) -> pl.DataFrame:
        return self.revenue().with_columns(
            (pl.col("Revenue") * self.ebitda_margin).alias("EBITDA")
        )

    # Assuming amortization is constant for now?
    def ebit(self) -> pl.DataFrame:
        return self.ebitda().with_columns((pl.col("EBITDA") - self.da).alias("EBIT"))

    def int_exp(self) -> pl.DataFrame:
        int_exp = debt * self.int_rate / 12
        return self.ebit().with_columns(pl.lit(int_exp).alias("Interest Expense"))

    def ebt(self) -> pl.DataFrame:
        return self.int_exp().with_columns(
            (pl.col("EBIT") - pl.col("Interest Expense")).alias("EBT")
        )

    def tax(self) -> pl.DataFrame:
        return self.ebt().with_columns(
            (pl.col("EBT") * self.tax_rate).alias("Tax Payable")
        )

    def net_income(self) -> pl.DataFrame:
        return self.tax().with_columns(
            (pl.col("EBT") - pl.col("Tax Payable")).alias("Net Income")
        )

In [132]:
# Creating an Income Statement instance
lbo_is = IncomeStatement(
    rev_y1,
    rev_growth,
    ebitda_margin,
    int_rate,
    tax_rate,
    d_and_a,
    debt,
    total_months,
).net_income()

lbo_is = lbo_is.with_columns(pl.exclude("Month").round(2))

# If you wish to go for the usual spreadsheet-like approach:
# lbo = lbo.with_columns(pl.exclude("Month").round(2)).transpose(include_header=True, header_name="Month", column_names="Month")
lbo_is

Month,Revenue,EBITDA,EBIT,Interest Expense,EBT,Tax Payable,Net Income
str,f64,f64,f64,f64,f64,f64,f64
"""Month 1""",8333333.33,3333333.33,1666666.67,83333.33,1583333.33,633333.33,950000
"""Month 2""",8333333.33,3333333.33,1666666.67,83333.33,1583333.33,633333.33,950000
"""Month 3""",8333333.33,3333333.33,1666666.67,83333.33,1583333.33,633333.33,950000
"""Month 4""",8333333.33,3333333.33,1666666.67,83333.33,1583333.33,633333.33,950000
"""Month 5""",8333333.33,3333333.33,1666666.67,83333.33,1583333.33,633333.33,950000
"""Month 6""",8333333.33,3333333.33,1666666.67,83333.33,1583333.33,633333.33,950000
"""Month 7""",8333333.33,3333333.33,1666666.67,83333.33,1583333.33,633333.33,950000
"""Month 8""",8333333.33,3333333.33,1666666.67,83333.33,1583333.33,633333.33,950000
"""Month 9""",8333333.33,3333333.33,1666666.67,83333.33,1583333.33,633333.33,950000


### 4. CALCULATE CUMULATIVE LEVERED FREE CASH FLOW (FCF)


In [14]:
# Create the DataFrame for the Free Cash Flow

lbo_fcf = pd.DataFrame(
    data=None,
    index=["Earnings", "plus: D&A", "less: capex", "less: NWC", "FCF"],
    columns=years,
)
lbo_fcf

,1,2,3,4,5,6
Earnings,NaN,NaN,NaN,NaN,NaN,NaN
plus: D&A,NaN,NaN,NaN,NaN,NaN,NaN
less: capex,NaN,NaN,NaN,NaN,NaN,NaN
less: NWC,NaN,NaN,NaN,NaN,NaN,NaN
FCF,NaN,NaN,NaN,NaN,NaN,NaN


#### Preparing Free Cash Flow

    (NOTE: We do not need Year 6 capital expenditures, or Free Cash Flow for that matter,
     because EBITDA does not incorporate capex and because only FCF in Years 1-5 can be used to pay down debt.)

     Calcs step:

     1. Start with EBT (Tax-effected) and then add back non-cash expenses (D&A).
     2. Add D&A to EBT
     3. Subtract capital expenditures (Capex).
     4. Subtract the annual increase in operating working capital to get to Free Cash Flow (FCF).
     5. Calculate Cumulative Free Cash Flow during the life of the LBO by adding FCF year-to-year


In [15]:
lbo_fcf.loc["Earnings"] = lbo_is.loc["Earnings"]
lbo_fcf.loc["plus: D&A"] = -lbo_is.loc["less: D&A"]
lbo_fcf.loc["less: capex"] = lbo_is.loc["Rev"] * -is_ass["capex % of rev"]
lbo_fcf.loc["less: NWC"] = -is_ass["change_in_NWC"]
lbo_fcf.loc["FCF"] = lbo_fcf.loc[
    ["Earnings", "plus: D&A", "less: capex", "less: NWC"]
].sum()
lbo_fcf

,1,2,3,4,5,6
Earnings,4.8,7.2,9.84,12.744,15.9384,19.45224
plus: D&A,20,20,20,20,20,20
less: capex,-15.0,-16.5,-18.15,-19.965,-21.9615,-24.15765
less: NWC,-5,-5,-5,-5,-5,-5
FCF,4.8,5.7,6.69,7.779,8.9769,10.29459


In [16]:
# Calculate Cumulative Free Cash Flow during the life of the LBO by adding FCF year-to-year

cumulative_fcf = lbo_fcf[[1, 2, 3, 4, 5]].loc["FCF"].sum()
cumulative_fcf

33.94590000000002

### CALCULATE ENDING PURCHASE PRICE (EXIT VALUE) AND RETURNS

Cals steps:

        1. Calculate Total Enterprise Value (TEV) at Exit: Take Forward EBITDA at exit (Year 6 EBITDA) along
           with a 5.0x exit multiple to calculate Exit TEV.
        2. Calculate Net Debt at Exit (also known as Ending Debt):
           Beginning Debt – Debt Pay-down = Ending Debt.
           Beginning Debt – Cumulative FCF = Ending Debt

        3. Calculate ending Equity Value (EV) by subtracting Ending Debt from Exit TEV.
        4. Calculate the Multiple-of-Money (MoM) EV return (Ending EV ÷ Beginning EV).


In [17]:
exit_returns = {
    "exit_EBITDA": lbo_is[6].loc["EBITDA"],
    "exit_multiple": entry_ass["entry_multiple"],
    "beginning_debt": entry_ass["debt_portion"],
    "cumulative_FCF": cumulative_fcf,
}
exit_returns

{'exit_EBITDA': 64.42040000000004,
 'exit_multiple': 5,
 'beginning_debt': 120.0,
 'cumulative_FCF': 33.94590000000002}

In [18]:
exit_returns["exit_TEV"] = exit_returns["exit_EBITDA"] * exit_returns["exit_multiple"]
exit_returns["ending_debt"] = (
    exit_returns["beginning_debt"] - exit_returns["cumulative_FCF"]
)
exit_returns["ending_equity"] = exit_returns["exit_TEV"] - exit_returns["ending_debt"]
exit_returns["MoM"] = exit_returns["ending_equity"] / entry_ass["equity_portion"]
exit_returns

{'exit_EBITDA': 64.42040000000004,
 'exit_multiple': 5,
 'beginning_debt': 120.0,
 'cumulative_FCF': 33.94590000000002,
 'exit_TEV': 322.1020000000002,
 'ending_debt': 86.05409999999998,
 'ending_equity': 236.04790000000023,
 'MoM': 2.950598750000003}

### Estimate IRR based on the MoM multiple.

    The following table is useful for estimating IRR based upon 5-year MoM multiples:
    2.0x MoM over 5 years ~15% IRR
    2.5x MoM over 5 years ~20% IRR
    3.0x MoM over 5 years ~25% IRR
    3.7x MoM over 5 years ~30% IRR

    Therefore, we can assume that the implied IRR for the paper LBO case study is approximately 25%, or slightly below
